## 準備をしよう！

### 1. ライブラリのインポート
- まずはじめに、このノートブックで使用するライブラリをインポートします。

### 2. DifyからAPIキーを取得
- Difyにログイン(https://dify-llm-study-2nd-50086289502.asia-northeast1.run.app/apps)
- ChatBotを作成
- 「APIアクセス」からAPIキーとURLを取得

### 3. 取得したAPIkeyとURLを設定
- 取得したAPIキーとURLをGoogle Colabに設定します。

### 4. そして以下を実行する。


In [ ]:

dify_api_key="your api key"
dify_base_url="your base url"

## 演習2

演習1の内容をベースにReActベースのエージェントを作成してみましょう。
以下のURLを参考にしてください。

- create_react_agentのドキュメント
https://langchain-ai.github.io/langgraph/reference/prebuilt/

- ReActを使ったエージェントの作成チュートリアル
https://python.langchain.com/docs/tutorials/agents/

#### (参考) 現在の日時を取得するツール

In [22]:


from langchain.tools import tool
from datetime import datetime

@tool("get_current_datetime")
def get_current_datetime() -> str:
    """
    現在の日時を取得する関数型ツール。
    :return: 現在の日時
    """
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"現在の日時は {current_time}"


### ツールを設定する

In [ ]:

tools = [get_current_datetime]

### 使用するLLMモデルを設定

In [ ]:
from chemcrow.agents.dify_llm import DifyCustomLLM
model =DifyCustomLLM(api_key=dify_api_key, user_id="llm-study-2", base_url=dify_base_url)

### 課題1：ReActベースのエージェントを作成
以下のURLを参考にしてReActでエージェントを作成してください。

- create_react_agentのドキュメント
https://langchain-ai.github.io/langgraph/reference/prebuilt/

- ReActを使ったエージェントの作成チュートリアル
https://python.langchain.com/docs/tutorials/agents/


In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful AI assistant. """),
    ("placeholder", "{messages}"),
])

def _modify_state_messages(state: AgentState):
    return prompt.invoke({"messages": state["messages"]}).to_messages()

query="What is today's date？"

# ここにエージェントの作成する
langgraph_agent_executor = 



# エージェントの出力を表示をここに書く




    # 出力を綺麗にしたい場合は以下のコメントアウトを外してください
    # if 'agent' in step:
    #     print("\n=== Agent Action ===")
    #     messages = step['agent']['messages'][0]
        
    #     # ツール呼び出しの情報を表示
    #     if messages.additional_kwargs.get('tool_calls'):
    #         for tool_call in messages.additional_kwargs['tool_calls']:
    #             print(f"Tool: {tool_call['function']['name']}")
    #             print(f"Arguments: {tool_call['function']['arguments']}")
        
    #     # 最終的な応答を表示
    #     if messages.content:
    #         print(f"Final Response: {messages.content}")
            
    # if 'tools' in step:
    #     print("\n=== Tool Response ===")
    #     for message in step['tools']['messages']:
    #         print(f"Tool Name: {message.name}")
    #         print(f"Result: {message.content}")
    
    # print("\n-------------------")

## 演習2+α

Google検索を行うツールを用意しています。
ReActベースのエージェントにGoogle検索を組み込んで「最新のAIニュース」を取得させてみましょう！

#### (参考)Google検索ツール

In [23]:
from typing import List, Optional
from pydantic import BaseModel
from langchain.tools import tool
import requests
from bs4 import BeautifulSoup

# 入力スキーマの定義
class GoogleSearchInput(BaseModel):
    query: str
    start_date: Optional[str] = None  # YYYY-MM-DD形式
    end_date: Optional[str] = None    # YYYY-MM-DD形式
    num_results: int = 10

@tool("google_search", args_schema=GoogleSearchInput)
def google_search_tool(query: str, start_date: Optional[str] = None, end_date: Optional[str] = None, num_results: int = 10) -> List[str]:
    """
    Google検索を実行し、指定されたクエリと日付範囲で結果を取得します。
    """
    # 日付フィルタのクエリを追加
    if start_date or end_date:
        if start_date:
            query += f" after:{start_date}"
        if end_date:
            query += f" before:{end_date}"

    print(f"検索クエリ: {query}")

    # Google検索のURL
    url = f"https://www.google.com/search?q={query}&num={num_results}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }

    # HTTPリクエスト
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return [f"エラー: Googleからの応答が不正です (HTTP {response.status_code})"]

    # BeautifulSoupでHTML解析
    soup = BeautifulSoup(response.text, "html.parser")
    results = []

    # 検索結果を抽出
    for g in soup.select(".tF2Cxc"):
        title = g.select_one(".DKV0Md").text if g.select_one(".DKV0Md") else "タイトルなし"
        link = g.select_one(".yuRUbf a")["href"] if g.select_one(".yuRUbf a") else "リンクなし"
        results.append(f"{title}: {link}")

        # 最大数に達したら終了
        if len(results) >= num_results:
            break

    return results


### 課題2：使用するツールを設定
以下のコードにgoogle_search_toolの設定を追加してみてください。

In [24]:

tools = [get_current_datetime, ]

### 課題3：ReActベースのエージェントを作成する。
1. 課題2で設定したツールを使ってReActベースのエージェントを作成してください。
2. promptをチューニングして、正しくAIニュースを取得できるようにしてください。

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a news search assistant. """),
    ("placeholder", "{messages}"),
])

def _modify_state_messages(state: AgentState):
    return prompt.invoke({"messages": state["messages"]}).to_messages()

query="Tell me about AI news"

# ここにエージェントの作成する
langgraph_agent_executor = 



# エージェントの出力を表示をここに書く




    # 出力を綺麗にしたい場合は以下のコメントアウトを外してください
    # if 'agent' in step:
    #     print("\n=== Agent Action ===")
    #     messages = step['agent']['messages'][0]
        
    #     # ツール呼び出しの情報を表示
    #     if messages.additional_kwargs.get('tool_calls'):
    #         for tool_call in messages.additional_kwargs['tool_calls']:
    #             print(f"Tool: {tool_call['function']['name']}")
    #             print(f"Arguments: {tool_call['function']['arguments']}")
        
    #     # 最終的な応答を表示
    #     if messages.content:
    #         print(f"Final Response: {messages.content}")
            
    # if 'tools' in step:
    #     print("\n=== Tool Response ===")
    #     for message in step['tools']['messages']:
    #         print(f"Tool Name: {message.name}")
    #         print(f"Result: {message.content}")
    
    # print("\n-------------------")